Code to genearte an NLP Based search engine on specific documents 

Data is from Kaggle on products and the code combined several fields as descriptor for the product

Then the NLP process ahave a search option to produce the closest matches. 

In [1]:
import kagglehub
import os
import pandas as pd 
import nltk
# Download NLTK data (run once)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('names')
# Download latest version of this dataset: https://www.kaggle.com/datasets/sujaykapadnis/products-datasets
path = kagglehub.dataset_download("sujaykapadnis/products-datasets")
documents = {}
print("Path to dataset files:", path)
try: 
    prod_data = pd.read_csv(path+os.listdir(path)[0])
    for index, row in prod_data.iterrows():
        documents[row['S.No']] = row['Product Name']+" "+row['Brand Desc']+" "+row['Product Size']+" "+row['Category']
except:
    path = '/Users/jorgepinzon/.cache/kagglehub/datasets/sujaykapadnis/products-datasets/versions/1/'
    prod_data = pd.read_csv(path+os.listdir(path)[0])
    for index, row in prod_data.iterrows():
        documents[row['S.No']] = row['Product Name']+" "+row['Brand Desc']+" "+row['Product Size']+" "+row['Category']
len(documents)
# Download latest version of this dataset: https://www.kaggle.com/datasets/sujaykapadnis/products-datasets
path2 = kagglehub.dataset_download("promptcloud/amazon-product-dataset-2020")

print("Path to dataset files:", path2)
try: 
    prod_data = pd.read_csv(path2+'/home/sdf/'+ os.listdir(path2+'/home/sdf/')[0])
    for index, row in prod_data.iterrows():
       documents[row['Uniq Id']] = str(row['Product Name'])+" "+str(row['Category'])
except:
    path2 = '/Users/jorgepinzon/.cache/kagglehub/datasets/promptcloud/amazon-product-dataset-2020/versions/1/home/sdf/marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv'
    prod_data = pd.read_csv(path2)
    for index, row in prod_data.iterrows():
        documents[row['Uniq Id']] = str(row['Product Name'])+" "+str(row['Category'])
len(documents)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jorgepinzon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jorgepinzon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     /Users/jorgepinzon/nltk_data...
[nltk_data]   Package names is already up-to-date!


Path to dataset files: /Users/jorgepinzon/.cache/kagglehub/datasets/sujaykapadnis/products-datasets/versions/1
Path to dataset files: /Users/jorgepinzon/.cache/kagglehub/datasets/promptcloud/amazon-product-dataset-2020/versions/1


14568

In [3]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords, wordnet, names
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell
import pkg_resources
import re
import torch

# Initialize tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Initialize SymSpell for spell-checking
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Load male and female names
male_names = set(names.words('male.txt'))
female_names = set(names.words('female.txt'))

# Text preprocessing
def preprocess(text):
    # Lowercase and remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    
    # Tokenize
    tokens = text.split()
    
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

# Spell-checking
def spell_check(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term  # Return the top suggestion

# Query expansion with synonyms
def expand_query_with_synonyms(query):
    tokens = query.split()
    expanded_query = []
    for token in tokens:
        expanded_query.append(token)
        synonyms = set()
        for syn in wordnet.synsets(token):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        expanded_query.extend(list(synonyms)[:2])  # Add top 2 synonyms
    return ' '.join(expanded_query)

# Gender differentiation
def infer_gender(text):
    # Tokenize and tag parts of speech
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    
    # Extract pronouns and names
    pronouns = [word.lower() for word, pos in pos_tags if pos == 'PRP']
    names_in_text = [word for word, pos in pos_tags if pos == 'NNP']
    
    # Count gender-specific pronouns
    gender_pronouns = {
        "male": ["he", "him", "his"],
        "female": ["she", "her", "hers"]
    }
    gender_counts = {gender: sum(1 for p in pronouns if p in pronouns_list) for gender, pronouns_list in gender_pronouns.items()}
    
    # Count gender-specific names
    gender_counts["male"] += sum(1 for name in names_in_text if name in male_names)
    gender_counts["female"] += sum(1 for name in names_in_text if name in female_names)
    
    # Infer gender
    inferred_gender = max(gender_counts, key=gender_counts.get) if max(gender_counts.values()) > 0 else "unknown"
    return inferred_gender

# Sample documents

# Preprocess documents
processed_docs = [preprocess(doc) for doc in documents.values()]

# Tokenize documents for BM25
tokenized_docs = [doc.split() for doc in processed_docs]

# Initialize BM25
bm25 = BM25Okapi(tokenized_docs)

# Load Sentence-BERT model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Hybrid search function with gender-aware ranking
def hybrid_search(query, top_k=5, bm25_weight=0.5, sbert_weight=0.5, gender_weight=0.2):
    # Step 1: Spell-check the query
    corrected_query = spell_check(query)
    print(f"Corrected Query: {corrected_query}")
    
    # Step 2: Infer gender for the query
    query_gender = infer_gender(corrected_query)
    print(f"Inferred Gender for Query: {query_gender}")
    
    # Step 3: Expand the query with synonyms
    expanded_query = expand_query_with_synonyms(corrected_query)
    print(f"Expanded Query: {expanded_query}")
    
    # Preprocess the expanded query
    processed_query = preprocess(expanded_query)
    
    # Step 4: BM25 for candidate retrieval
    query_tokens = processed_query.split()
    bm25_scores = bm25.get_scores(query_tokens)
    
    # Get top-k candidates based on BM25 scores
    candidate_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    candidate_docs = [processed_docs[i] for i in candidate_indices]
    candidate_ids = [list(documents.keys())[i] for i in candidate_indices]
    
    # Step 5: Sentence-BERT for re-ranking
    candidate_embeddings = model.encode(candidate_docs, device=device)
    query_embedding = model.encode([processed_query], device=device)
    sbert_scores = cosine_similarity(query_embedding, candidate_embeddings).flatten()
    
    # Step 6: Infer gender for candidate documents
    candidate_genders = [infer_gender(documents[doc_id]) for doc_id in candidate_ids]
    
    # Step 7: Combine BM25, Sentence-BERT, and gender scores
    combined_scores = []
    for j, i in enumerate(candidate_indices):
        # Base score from BM25 and Sentence-BERT
        base_score = (bm25_weight * bm25_scores[i]) + (sbert_weight * sbert_scores[j])
        
        # Gender score: boost if document gender matches query gender
        gender_score = 1.0 if candidate_genders[j] == query_gender else 0.0
        
        # Combined score
        combined_score = base_score + (gender_weight * gender_score)
        combined_scores.append(combined_score)
    
    # Rank documents by combined scores
    ranked_docs = sorted(zip(candidate_ids, combined_scores, candidate_genders), key=lambda x: x[1], reverse=True)
    return ranked_docs

/opt/anaconda3/envs/p312/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Example query
query = '''Can you suggest a present for alice? '''
results = hybrid_search(query, top_k=5, bm25_weight=1, sbert_weight=1)
# Print results
print("\nRanked Documents:")
for doc_id, score, gender in results:
    print(f"Document: {doc_id}, Score: {score:.4f}, Inferred Gender: {gender}")
    print(f'>>> {documents[doc_id]}')

Corrected Query: can you suggest a present for alice
Inferred Gender for Query: unknown
Expanded Query: can force_out fundament you suggest hint evoke a angstrom deoxyadenosine_monophosphate present demo portray for alice

Ranked Documents:
Document: 435, Score: 9.9272, Inferred Gender: female
>>> AJMAL18 - Fragrance-Women mens evoke silver edithimclassi deodorant 200 ml Nan Fragrance-Women
Document: bcb528aab71214f7c240b4a695ae6415, Score: 9.6757, Inferred Gender: female
>>> Charades Storybook Alice Children's Costume, Small Toys & Games | Dress Up & Pretend Play | Costumes
Document: 428, Score: 9.2900, Inferred Gender: female
>>> AJMAL11 - Fragrance-Women womens evoke gold edition her edp fruity perfume - 75 ml Nan Fragrance-Women
Document: 427, Score: 9.2691, Inferred Gender: female
>>> AJMAL10 - Fragrance-Women womens evoke silver edition her edp citrus perfume - 75 ml Nan Fragrance-Women
Document: 9eb3927f946ce5b4ca4b57630f7fadbc, Score: 8.1012, Inferred Gender: unknown
>>> 2.4Ghz

In [ ]:
# WITH OUT CHECKING FOR SYNONIMS 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK stopwords (run once)



def preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# # Sample documents
# documents = {
#     "doc1": "Python is a programming language",
#     "doc2": "Python is used for web development and data science",
#     "doc3": "Natural Language Processing powers search engines",
# }

# Preprocess docs
processed_docs = [preprocess(doc) for doc in documents.values()]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_docs)

# Search function
def search(query):
    processed_query = preprocess(query)
    query_vec = vectorizer.transform([processed_query])
    cos_sim = cosine_similarity(query_vec, tfidf_matrix)
    scores = cos_sim.flatten()
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]



In [ ]:
ranked_docs, results = search("cozzy shirt")
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

In [ ]:
import re
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from symspellpy import SymSpell
import pkg_resources

# Initialize SymSpell for spell-check
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Text preprocessing
def syn_preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Spell-check
def spell_check(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term

# Synonyms
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def expand_query_with_synonyms(query):
    tokens = query.split()
    expanded_query = []
    for token in tokens:
        expanded_query.append(token)
        synonyms = get_synonyms(token)
        expanded_query.extend(synonyms[:2])  # Add top 2 synonyms
    return ' '.join(expanded_query)

# Preprocess docs
processed_docs = [syn_preprocess(doc) for doc in documents.values()]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_docs)

# Search function
def syn_search(query):
    # Spell-check
    corrected_query = spell_check(query)
    print(f"Corrected Query: {corrected_query}")
    
    # Expand with synonyms
    expanded_query = expand_query_with_synonyms(corrected_query)
    print(f"Expanded Query: {expanded_query}")
    
    # Preprocess query
    processed_query = syn_preprocess(expanded_query)
    
    # Vectorize and compare
    query_vec = vectorizer.transform([processed_query])
    cos_sim = cosine_similarity(query_vec, tfidf_matrix)
    scores = cos_sim.flatten()
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score >0]

In [ ]:
scores, results = syn_search("I need a warm t-shirt for a boy, not a perfume, not too big but also not small")
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

In [ ]:
for doc, score in scores:
    if score > 0.1:
        print(f"Document: {doc}, Score: {score:.2f}")

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from symspellpy import SymSpell
import pkg_resources
import re
from rank_bm25 import BM25Okapi


# Initialize tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Preprocessing
def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Spell-check
def spell_check(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term

# Query expansion
def expand_query_with_synonyms(query):
    tokens = query.split()
    expanded_query = []
    for token in tokens:
        expanded_query.append(token)
        synonyms = set()
        for syn in wordnet.synsets(token):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        expanded_query.extend(list(synonyms)[:2])
    return ' '.join(expanded_query)
# Preprocess documents
processed_docs = [preprocess(doc) for doc in documents.values()]

# TF-IDF Vectorization with n-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(processed_docs)

#BM25
# Tokenize documents
tokenized_docs = [doc.split() for doc in processed_docs]
# Initialize BM25
bm25 = BM25Okapi(tokenized_docs)
# BERT: 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
# Load pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and fast model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Encode documents in batches
def encode_in_batches(texts, model, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, device=device)
        embeddings.extend(batch_embeddings)
    return embeddings

doc_embeddings = encode_in_batches(processed_docs, model, batch_size=32)

# Search function
def impr_search(query):
    # Spell-check
    corrected_query = spell_check(query)
    print(f"Corrected Query: {corrected_query}")
    
    # Expand with synonyms
    expanded_query = expand_query_with_synonyms(corrected_query)
    print(f"Expanded Query: {expanded_query}")
    
    # Preprocess query
    processed_query = preprocess(expanded_query)
    
    # Vectorize and compare
    query_vec = vectorizer.transform([processed_query])
    cos_sim = cosine_similarity(query_vec, tfidf_matrix)
    scores = cos_sim.flatten()
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]

def bmk_search(query):
    # Preprocess query
    processed_query = preprocess(query)
    query_tokens = processed_query.split()
    
    # Get BM25 scores
    scores = bm25.get_scores(query_tokens)
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]

def bert_search(query):
    # Preprocess query
    processed_query = preprocess(query)
    
    # Encode query
    query_embedding = model.encode([processed_query], device=device)
    
    # Compute cosine similarity
    scores = cosine_similarity(query_embedding, doc_embeddings).flatten()
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]

In [ ]:
sentence = '''Can you suggest a nice cologne for my son? '''

In [ ]:
ranked_docs, results = impr_search(sentence)
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

In [ ]:
ranked, results = bmk_search(sentence)
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

In [ ]:
ranked, results = bert_search(sentence)
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

In [ ]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell
import pkg_resources
import re
import nltk

# Download NLTK data (run once)
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Initialize SymSpell for spell-checking
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Text preprocessing
def preprocess(text):
    # Lowercase and remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    
    # Tokenize
    tokens = text.split()
    
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

# Spell-checking
def spell_check(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term  # Return the top suggestion

# Query expansion with synonyms
def expand_query_with_synonyms(query):
    tokens = query.split()
    expanded_query = []
    for token in tokens:
        expanded_query.append(token)
        synonyms = set()
        for syn in wordnet.synsets(token):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        expanded_query.extend(list(synonyms)[:2])  # Add top 2 synonyms
    return ' '.join(expanded_query)

# Sample documents
# documents = {
#     "doc1": "Python is a programming language",
#     "doc2": "Python is used for web development and data science",
#     "doc3": "Natural Language Processing powers search engines",
#     "doc4": "Data science involves Python and machine learning",
#     "doc5": "Web development requires HTML, CSS, and JavaScript",
# }

# Preprocess documents
processed_docs = [preprocess(doc) for doc in documents.values()]

# Tokenize documents for BM25
tokenized_docs = [doc.split() for doc in processed_docs]

# Initialize BM25
bm25 = BM25Okapi(tokenized_docs)

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight model

# Hybrid search function
def hybrid_search(query, top_k=5, bm25_weight=0.5, sbert_weight=0.5):
    # Step 1: Spell-check the query
    corrected_query = spell_check(query)
    print(f"Corrected Query: {corrected_query}")
    
    # Step 2: Expand the query with synonyms
    expanded_query = expand_query_with_synonyms(corrected_query)
    print(f"Expanded Query: {expanded_query}")
    
    # Preprocess the expanded query
    processed_query = preprocess(expanded_query)
    
    # Step 3: BM25 for candidate retrieval
    query_tokens = processed_query.split()
    bm25_scores = bm25.get_scores(query_tokens)
    
    # Get top-k candidates based on BM25 scores
    candidate_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    candidate_docs = [processed_docs[i] for i in candidate_indices]
    candidate_ids = [list(documents.keys())[i] for i in candidate_indices]
    
    # Step 4: Sentence-BERT for re-ranking
    candidate_embeddings = model.encode(candidate_docs)
    query_embedding = model.encode([processed_query])
    sbert_scores = cosine_similarity(query_embedding, candidate_embeddings).flatten()
    
    # Step 5: Combine BM25 and Sentence-BERT scores
    combined_scores = [
        (bm25_weight * bm25_scores[i]) + (sbert_weight * sbert_scores[j])
        for j, i in enumerate(candidate_indices)
    ]
    
    # Rank documents by combined scores
    ranked_docs = sorted(zip(candidate_ids, combined_scores), key=lambda x: x[1], reverse=True)
    return ranked_docs

# Example query
query = "Pythn data scince"
results = hybrid_search(query, top_k=5, bm25_weight=0.5, sbert_weight=0.5)

# Print results
print("\nRanked Documents:")
for doc_id, score in results:
    print(f"Document: {doc_id}, Score: {score:.4f}")

In [ ]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell
import pkg_resources
import re
import nltk
import torch
import gc

# Download NLTK data (run once)
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Initialize SymSpell for spell-checking
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Text preprocessing
def preprocess(text):
    # Lowercase and remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    
    # Tokenize
    tokens = text.split()
    
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

# Spell-checking
def spell_check(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term  # Return the top suggestion

# Query expansion with synonyms
def expand_query_with_synonyms(query):
    tokens = query.split()
    expanded_query = []
    for token in tokens:
        expanded_query.append(token)
        synonyms = set()
        for syn in wordnet.synsets(token):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        expanded_query.extend(list(synonyms)[:2])  # Add top 2 synonyms
    return ' '.join(expanded_query)
# Preprocess documents
processed_docs = [preprocess(doc) for doc in documents.values()]

# Tokenize documents for BM25
tokenized_docs = [doc.split() for doc in processed_docs]

# Initialize BM25
bm25 = BM25Okapi(tokenized_docs)

# Load Sentence-BERT model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Encode documents in batches
def encode_in_batches(texts, model, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, device=device)
        embeddings.extend(batch_embeddings)
    return embeddings

# Free up memory
gc.collect()
torch.cuda.empty_cache()

# Encode documents
doc_embeddings = encode_in_batches(processed_docs, model, batch_size=32)

# Hybrid search function
def hybrid_search(query, top_k=5, bm25_weight=0.5, sbert_weight=0.5):
    # Step 1: Spell-check the query
    corrected_query = spell_check(query)
    print(f"Corrected Query: {corrected_query}")
    
    # Step 2: Expand the query with synonyms
    expanded_query = expand_query_with_synonyms(corrected_query)
    print(f"Expanded Query: {expanded_query}")
    
    # Preprocess the expanded query
    processed_query = preprocess(expanded_query)
    
    # Step 3: BM25 for candidate retrieval
    query_tokens = processed_query.split()
    bm25_scores = bm25.get_scores(query_tokens)
    
    # Get top-k candidates based on BM25 scores
    candidate_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    candidate_docs = [processed_docs[i] for i in candidate_indices]
    candidate_ids = [list(documents.keys())[i] for i in candidate_indices]
    
    # Step 4: Sentence-BERT for re-ranking
    candidate_embeddings = model.encode(candidate_docs, device=device)
    query_embedding = model.encode([processed_query], device=device)
    sbert_scores = cosine_similarity(query_embedding, candidate_embeddings).flatten()
    
    # Step 5: Combine BM25 and Sentence-BERT scores
    combined_scores = [
        (bm25_weight * bm25_scores[i]) + (sbert_weight * sbert_scores[j])
        for j, i in enumerate(candidate_indices)
    ]
    
    # Rank documents by combined scores
    ranked_docs = sorted(zip(candidate_ids, combined_scores), key=lambda x: x[1], reverse=True)
    return ranked_docs

# Example query
query = "Pythn data scince"
query = '''Can you suggest a nice cologne for my son? '''
results = hybrid_search(query, top_k=5, bm25_weight=0.5, sbert_weight=0.5)

# Print results
print("\nRanked Documents:")
for doc_id, score in results:
    print(f"Document: {doc_id}, Score: {score:.4f}")

In [ ]:
for doc_id, score in results:
    print(f"Document: {doc_id}, Score: {score:.4f}")
    print(documents[doc_id])